# About this notebook 

#### Feature: Gender

This is one of a series of notebooks (one for each feature of interest) that explores the feature for missing data, data characteristics, correlation with the rate of adoption (predictor variable) and other points of interest that might be helpful to know (and deal with) prior to machine learning.

<div class="span5 alert alert-success">
<p> <I> Feature Description: </I> The "Gender" data represents whether the pet is a male (value of 1), a female (value of 2) or a mix of several pets (value of 3).   
         <br>
    <I> Source: </I> https://www.kaggle.com/c/petfinder-adoption-prediction/data  </p>
</div>

<div class="span5 alert alert-success">
<p> <I> Predictor (Adoption Speed) Description: </I> 

Contestants are required to predict this value. The value is determined by how quickly, if at all, a pet is adopted.   
<br> 
The values are determined in the following way:   
0 - Pet was adopted on the same day as it was listed.    
1 - Pet was adopted between 1 and 7 days (1st week) after being listed.    
2 - Pet was adopted between 8 and 30 days (1st month) after being listed.    
3 - Pet was adopted between 31 and 90 days (2nd & 3rd month) after being listed.    
4 - No adoption after 100 days of being listed.    

</p>
</div>

In [1]:
import warnings
warnings.filterwarnings('ignore')

%cd C:\Users\Ken\Documents\KenP\Applications-DataScience\SpringboardCourseWork\CapstoneProject2Repository\09 PetfindersData\TrainingData

C:\Users\Ken\Documents\KenP\Applications-DataScience\SpringboardCourseWork\CapstoneProject2Repository\09 PetfindersData\TrainingData


<div class="span5 alert alert-info">
<p> <B>  Imports and Data Loading: </B>  </p>
</div>

In [2]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#Import the csv file

dfi = pd.read_csv('train.csv')
dfi.head(1)

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,1,Lil Milo,2,0,26,2,2,0,0,2,...,1,1,0,41326,1a2113010d6048d5410b265347b35c91,0,Milo went missing after a week with her new ad...,375905770,3,3


<div class="span5 alert alert-info">
<p> <B>  Missing Data: </B>  </p>
</div>

In [4]:
#Create Type Dataframe

dfa = dfi[['Gender','AdoptionSpeed']]
dfa.columns = ['gender','adoptionspeed']

In [5]:
# Percentage of missing values in each column
pd.DataFrame(
    dfa.isnull().sum() / len(dfa),
    columns=['% Missing Values']
).transpose()

,gender,adoptionspeed
% Missing Values,0.0,0.0


<div class="span5 alert alert-info">
<p> <B>  Characteristics of the data: </B>  </p>
</div>

In [6]:
#Function to designate 'male', 'female', or 'mix' in dataframe
def petgender(intype):
    petsgender = ''
    
    if intype == 1:
        petsgender = 'male'
    elif intype == 2:
        petsgender = 'female'
    else:
        petsgender = 'mix'
    
    return petsgender

In [7]:
#Create a dataframe of genders

dfac = dfa['gender'].value_counts()
dfac = dfac.reset_index()
dfac.columns = ['gender','gendercount']

dfac['gendertext'] = dfac.gender.apply(petgender)

dfac['genderpercent'] = round(dfac.gendercount/dfac.gendercount.sum(),2)

dfac = dfac[['gender','gendertext','gendercount','genderpercent']]
dfac

,gender,gendertext,gendercount,genderpercent
0,2,female,7277,0.49
1,1,male,5536,0.37
2,3,mix,2180,0.15


In [8]:
#Average adoption speed for a male
dfmales = dfa[dfa.gender == 1]

malesavgadoptionspeed = round(dfmales.adoptionspeed.sum()/dfmales.gender.count(),2)
print('males average adoption speed: ' + str(malesavgadoptionspeed))


males average adoption speed: 2.42


In [9]:
#Average adoption speed for a female
dffemales = dfa[dfa.gender == 2]

femalesavgadoptionspeed = round(dffemales.adoptionspeed.sum()/dffemales.gender.count(),2)
print('females average adoption speed: ' + str(femalesavgadoptionspeed))


females average adoption speed: 2.57


In [10]:
#Average adoption speed for a mix
dfmix = dfa[dfa.gender == 2]

mixavgadoptionspeed = round(dfmix.adoptionspeed.sum()/dfmix.gender.count(),2)
print('mix average adoption speed: ' + str(mixavgadoptionspeed))


mix average adoption speed: 2.57


<div class="span5 alert alert-info">
<p> <B>  Correlation with the Adoption Rate: </B>  </p>
</div>

In [11]:
#Create a dataframe to calculate correlation
dfaa = dfi[['Gender','AdoptionSpeed']]
dfaa.columns = ['gender','adoptionspeed']

In [12]:
#Calculate pearson correlation between pet type and adoption speed
def pearson_r(x,y):
    corr_mat = np.corrcoef(x,y)

    return corr_mat[0,1]

# Compute Pearson correlation coefficient
r = pearson_r(dfaa.gender,dfaa.adoptionspeed)

# Print the result
print('Correlation value: ' + str(round(r,2)*100) + '%')

Correlation value: 6.0%


<div class="span5 alert alert-info">
<p> <B>  Other Points of Interest: </B>  </p>
</div>

In [13]:
#Create a dataframe for pivot table for gender
dfaa = dfi[['Gender','Age','AdoptionSpeed']]
dfaa['ageinyears'] = round(dfaa.Age/12)
dfaa.columns = ['gender','ageinmonths','adoptionspeed','ageinyears']
dfaa = dfaa[dfaa.ageinyears < 13]
dfaa = dfaa[['gender','ageinyears','adoptionspeed','ageinmonths']]

In [14]:
#Create a pivot table of adoption speed vs age for dogs and cats
#dfaas = dfaa[['ageinyears','adoptionspeed']]
dfasa = dfaa.pivot_table(columns='adoptionspeed', index=['ageinyears','gender'], values='ageinmonths', aggfunc='count',margins=True)
dfasa.columns = ['oneday','oneweek','onemonth','threemonths','notadopted','totals']

dfasa['%adoptedinoneday'] = round(dfasa.oneday/dfasa.totals,2)
dfasa['%notadopted'] = round(dfasa.notadopted/dfasa.totals,2)
dfasa

oneday  oneweek  onemonth  threemonths  notadopted  totals  \
ageinyears gender                                                               
0.0        1        101.0    947.0    1125.0        697.0       629.0    3499   
           2        136.0   1011.0    1419.0       1172.0      1088.0    4826   
           3         37.0    399.0     499.0        432.0       522.0    1889   
1.0        1         34.0    134.0     212.0        181.0       390.0     951   
           2         31.0    155.0     250.0        250.0       517.0    1203   
           3          3.0     23.0      27.0         20.0        82.0     155   
2.0        1         12.0     76.0      69.0        102.0       157.0     416   
           2         17.0     85.0      99.0        111.0       252.0     564   
           3          6.0     14.0      11.0         13.0        35.0      79   
3.0        1          3.0     47.0      42.0         37.0        86.0     215   
           2          6.0     38.0      56.0         33.0       116.0     249   
           3          NaN      2.0       3.0          6.0        12.0      23   
4.0        1          2.0     33.0      41.0         30.0        49.0     155   
           2          4.0     30.0      26.0         29.0        65.0     154   
           3          NaN      1.0       4.0          1.0         7.0      13   
5.0        1          1.0     27.0      32.0         26.0        32.0     118   
           2          5.0     24.0      21.0         22.0        29.0     101   
           3          NaN      NaN       3.0          2.0         4.0       9   
6.0        1          4.0      4.0      22.0          9.0        23.0      62   
           2          1.0      5.0      16.0         23.0        14.0      59   
           3          NaN      1.0       1.0          1.0         2.0       5   
7.0        1          1.0      6.0      19.0         16.0        15.0      57   
           2          1.0      9.0      11.0         16.0        16.0      53   
           3          NaN      NaN       NaN          2.0         1.0       3   
8.0        1          1.0      3.0      10.0          4.0         7.0      25   
           2          2.0      5.0       4.0          6.0         9.0      26   
           3          NaN      NaN       NaN          1.0         NaN       1   
9.0        1          1.0      3.0       3.0          NaN         4.0      11   
           2          NaN      NaN       1.0          4.0         6.0      11   
           3          NaN      NaN       NaN          NaN         1.0       1   
10.0       1          NaN      3.0       2.0          2.0         8.0      15   
           2          1.0      2.0       6.0          2.0         9.0      20   
           3          NaN      1.0       NaN          NaN         NaN       1   
11.0       1          NaN      NaN       NaN          2.0         NaN       2   
           2          NaN      1.0       1.0          2.0         3.0       7   
12.0       1          NaN      NaN       1.0          1.0         2.0       4   
           2          NaN      1.0       NaN          NaN         NaN       1   
All                 410.0   3090.0    4036.0       3255.0      4192.0   14983   

                   %adoptedinoneday  %notadopted  
ageinyears gender                                 
0.0        1                   0.03         0.18  
           2                   0.03         0.23  
           3                   0.02         0.28  
1.0        1                   0.04         0.41  
           2                   0.03         0.43  
           3                   0.02         0.53  
2.0        1                   0.03         0.38  
           2                   0.03         0.45  
           3                   0.08         0.44  
3.0        1                   0.01         0.40  
           2                   0.02         0.47  
           3                    NaN         0.52  
4.0        1                   0.01         0.32  
           2 